In [1]:
import sys
sys.path.append('../')
import deepbayesHF
import deepbayesHF.optimizers as optimizers
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28* 28)

In [3]:
model = Sequential()
model.add(Dense(16, activation="relu", input_shape=(None, 28*28)))
model.add(Dense(10, activation="softmax"))
loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [4]:
learning_rate = 0.35; decay=0.0
opt = optimizers.VariationalOnlineGuassNewton()
bayes_model = opt.compile(model, loss_fn=loss, epochs=10, learning_rate=learning_rate, batch_size=128)
bayes_model.train(X_train, y_train, X_test, y_test)

This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.
deepbayes: Using implicit prior
(784, 16) 0.03571428571428571
(16, 10) 0.25
deepbayes: Using implicit prior
(784, 16) 0.03571428571428571
(16, 10) 0.25


  0%|                                                                                                          | 0/469 [00:00<?, ?it/s]/home/matker/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:08<00:00, 54.42it/s]


Epoch 1, loss: 0.951, acc: 0.067, val_loss: 0.662, val_acc: 0.070


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:07<00:00, 59.81it/s]


Epoch 2, loss: 0.684, acc: 0.067, val_loss: 0.636, val_acc: 0.068


...

Epoch 9, loss: 0.260, acc: 0.061, val_loss: 0.255, val_acc: 0.070


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:07<00:00, 59.80it/s]


Epoch 10, loss: 0.239, acc: 0.068, val_loss: 0.238, val_acc: 0.071


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [00:07<00:00, 60.48it/s]


Epoch 11, loss: 0.228, acc: 0.062, val_loss: 0.224, val_acc: 0.072


In [5]:
bayes_model.save("PosteriorModels/VOGN_MNIST_Posterior")

('classes', 10)
('batch_size', 128)
('learning_rate', 0.35)
('decay', 0.0)
('epochs', 11)
('inflate_prior', 1)
('input_noise', 0.0)
('robust_train', 0)
('epsilon', 0.10000000000000002)
('robust_lambda', 0.5)
('loss_monte_carlo', 2)
('input_upper', inf)
('input_lower', -inf)
('beta_1', 0.999)
('beta_2', 0.9999)
('lam', 1.0)
('N', 60000)
('max_eps', 0.1)
('max_robust_lambda', 0.5)


In [6]:
from deepbayesHF import PosteriorModel

bayes_model = PosteriorModel("PosteriorModels/VOGN_MNIST_Posterior")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, None, 16)          12560     
                                                                 
 dense_1 (Dense)             (None, None, 10)          170       
                                                                 
Total params: 12,730
Trainable params: 12,730
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 
 None


In [7]:
y_pred = bayes_model.predict(X_test)

In [8]:
import numpy as np
from tqdm import trange
y_cls = np.argmax(y_pred, axis=1)
corr = 0
for i in trange(len(y_cls)):
    corr += int(y_cls[i] == y_test[i])
print("Test set accuracy: ", corr/10000)

100%|████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 734451.22it/s]

Test set accuracy:  0.9344


In [48]:
# !! THIS IS ONLY FOR RELU !! NO OTHER ACTIVATION SUPPORTED
def get_alphas_betas(zeta_l, zeta_u, activation="relu"):
    zeta_l, zeta_u = np.squeeze(zeta_l), np.squeeze(zeta_u)
    alpha_L, alpha_U = list([]), list([])
    beta_L, beta_U = list([]), list([])
    print(zeta_l.shape)
    for i in range(len(zeta_l)):
        if(zeta_u[i] <= 0):
            alpha_U.append(0); alpha_L.append(0); beta_L.append(0); beta_U.append(0)
        elif(zeta_l[i] >= 0):
            alpha_U.append(1); alpha_L.append(1); beta_L.append(0); beta_U.append(0)
        else:
            # For relu I have the points (zeta_l, 0) and (zeta_u, zeta_u)
            a_U = zeta_u[i]/(zeta_u[i]-zeta_l[i]); b_U = -1*(a_U*zeta_l[i])
    
            #a_L = a_U ; b_L = 0
            #if (zeta_u[i] + zeta_l[i]) >= 0:
            #    a_L = 1 ;   b_L = 0
            #else:
            a_L = 0 ;   b_L = 0    
            alpha_U.append(a_U); alpha_L.append(a_L); beta_L.append(b_L); beta_U.append(b_U)
    return [alpha_U], [beta_U], [alpha_L], [beta_L]



def get_bar_lower(linear_bound_coef, mu_l, mu_u,
                  nu_l, nu_u, lam_l, lam_u):
    lam_l, lam_u = np.squeeze(lam_l), np.squeeze(lam_u)
    mu_l = np.squeeze(mu_l); mu_u = np.squeeze(mu_u); 
    #nu_l = np.squeeze(nu_l); nu_u = np.squeeze(nu_u); 
    #lam_l = np.squeeze(lam_l); lam_u = np.squeeze(lam_u); 
    #linear_bound_coef= np.squeeze(linear_bound_coef)
    mu_bar, nu_bar, lam_bar = [], [], []
    #coef of the form - alpha_U, beta_U, alpha_L, beta_L
    for i in range(len(linear_bound_coef)):
        if(linear_bound_coef[i,2] >= 0):
            mu_bar.append(linear_bound_coef[i,2] * mu_l[i])
            nu_bar.append(linear_bound_coef[i,2] * nu_l[i])
            lam_bar.append(linear_bound_coef[i,2] * lam_l[i] + linear_bound_coef[i,3])
        else:
            mu_bar.append(linear_bound_coef[i,2] * mu_u[i])
            nu_bar.append(linear_bound_coef[i,2] * nu_u[i])
            lam_bar.append(linear_bound_coef[i,2] * lam_u[i] + linear_bound_coef[i,3])
    return np.asarray(mu_bar), np.asarray(nu_bar), np.asarray(lam_bar)

def get_bar_upper(linear_bound_coef, mu_l, mu_u,
                  nu_l, nu_u, lam_l, lam_u):
    lam_l, lam_u = np.squeeze(lam_l), np.squeeze(lam_u)
    mu_l = np.squeeze(mu_l); mu_u = np.squeeze(mu_u);  
    #lam_l = np.squeeze(lam_l); lam_u = np.squeeze(lam_u); 
    mu_bar, nu_bar, lam_bar = [], [], []
    #coef of the form - alpha_U, beta_U, alpha_L, beta_L
    for i in range(len(linear_bound_coef)):
        if(linear_bound_coef[i,0] >= 0):
            mu_bar.append(linear_bound_coef[i,0] * mu_u[i])
            nu_bar.append(linear_bound_coef[i,0] * nu_u[i])
            lam_bar.append(linear_bound_coef[i,0] * lam_u[i] + linear_bound_coef[i,1])
        else:
            mu_bar.append(linear_bound_coef[i,0] * mu_l[i])
            nu_bar.append(linear_bound_coef[i,0] * nu_l[i])
            lam_bar.append(linear_bound_coef[i,0] * lam_l[i] + linear_bound_coef[i,1])
    return np.asarray(mu_bar), np.asarray(nu_bar), np.asarray(lam_bar)

def get_abc_lower(w, mu_l_bar, nu_l_bar, la_l_bar,
               mu_u_bar, nu_u_bar, la_u_bar):
    print("W shape: ", w.shape)
    print("Nu shape: ", nu_l_bar.shape, nu_u_bar.shape)
    a, b, c = [], [], []
    for i in range(len(w)):
        for j in range(len(w[i])):
            if(w[i][j] >= 0):
                a.append(w[i][j] * mu_l_bar[i])
                b.append(w[i][j] * nu_l_bar[i])
                c.append(w[i][j] * la_l_bar[i])
            else:
                a.append(w[i][j] * mu_u_bar[i])
                b.append(w[i][j] * nu_u_bar[i])
                c.append(w[i][j] * la_u_bar[i])
    return np.asarray(a), np.asarray(b), np.asarray(c)


def get_abc_upper(w, mu_l_bar, nu_l_bar, la_l_bar,
               mu_u_bar, nu_u_bar, la_u_bar):
    #This is anarchy
    return get_abc_lower(w,mu_u_bar, nu_u_bar, la_u_bar,
                         mu_l_bar, nu_l_bar, la_l_bar)


def min_of_linear_fun(coef_vec, uppers, lowers):
    print(coef_vec.shape, uppers.shape, lowers.shape)
    #getting the minimum
    val_min = 0
    for i in range(len(coef_vec)):
        if coef_vec[i] >=0:
            val_min = val_min + coef_vec[i]*lowers[i]
        else: 
            val_min = val_min + coef_vec[i]*uppers[i]
    return val_min

def max_of_linear_fun(coef_vec, uppers, lowers):
    val_max = - min_of_linear_fun(-coef_vec, uppers, lowers)
    return val_max

    
"""
Linear Propogation Function
"""

def my_relu(arr):
    arr = arr * (arr > 0)
    return arr

import pickle
def propogate_lines(model, x, in_reg, w_margin=0.25, act = 'relu'):
    x = np.asarray(x); x = x.astype('float64')
    x_l, x_u = in_reg[0], in_reg[1]
    [mW_0, mb_0, mW_1, mb_1] = model.posterior_mean
    [dW_0, db_0, dW_1, db_1] = model.posterior_var
    [sW_0, sb_0, sW_1, sb_1] = model.sample()
    
    [mW_0, mb_0, mW_1, mb_1] = [mW_0.numpy(), mb_0.numpy(), mW_1.numpy(), mb_1.numpy()]
    [dW_0, db_0, dW_1, db_1] = [dW_0.numpy(), db_0.numpy(), dW_1.numpy(), db_1.numpy()]
    #[sW_0, sb_0, sW_1, sb_1] = [sW_0.numpy(), sb_0.numpy(), sW_1.numpy(), sb_1.numpy()]
    
    #Actual code from now on    
    W_0_L, W_0_U, b_0_L, b_0_U = sW_0 - dW_0*w_margin,  sW_0 + dW_0*w_margin, sb_0-db_0*w_margin, sb_0+db_0*w_margin
    
    W_0_L = W_0_L.T
    W_0_U = W_0_U.T
    
    mu_0_L = W_0_L; mu_0_U = W_0_U
    
    print(sW_0[0])
    n_hidden_1 = len(sW_0[0]) 
    
    nu_0_L = np.asarray([x_l for i in range(n_hidden_1) ])
    nu_0_U = np.asarray([x_l for i in range(n_hidden_1) ])

    la_0_L = - np.dot(x_l, W_0_L.T) + b_0_L
    la_0_U = - np.dot(x_l, W_0_U.T) + b_0_U
    
    # getting bounds on pre-activation fucntion
    zeta_0_L = [ (min_of_linear_fun(np.concatenate((mu_0_L[i].flatten(), nu_0_L[i].flatten())), 
                                     np.concatenate((np.asarray(x_u).flatten(), W_0_U[i].flatten() )),
                                     np.concatenate((np.asarray(x_l).flatten(), W_0_U[i].flatten() ))  )) for i in range(n_hidden_1)] 
    zeta_0_L = np.asarray(zeta_0_L) + la_0_L
     
    zeta_0_U = [ (max_of_linear_fun(np.concatenate((mu_0_U[i].flatten(), nu_0_U[i].flatten())),
                                     np.concatenate((np.asarray(x_u).flatten(), W_0_U[i].flatten())),
                                     np.concatenate((np.asarray(x_l).flatten(), W_0_L[i].flatten()))  )) for i in range(n_hidden_1)]
        
    zeta_0_U = np.asarray(zeta_0_U) + la_0_U
    print("Zeta Shapes: ", zeta_0_L.shape, zeta_0_U.shape)
    
    # These are our linear upper and lower bounds for the activation function
    LUB = np.asarray(get_alphas_betas(zeta_0_L, zeta_0_U))
    #LUB = np.reshape(LUB, (-1, 4))
    LUB = np.asmatrix(LUB).transpose() 
    # Now evaluate eq (*) conditions:
    mu_0_L_bar, nu_0_L_bar, la_0_L_bar = get_bar_lower(LUB, mu_0_L, mu_0_U, 
                                                       nu_0_L, nu_0_U, 
                                                      la_0_L, la_0_U)
    nu_0_L_bar = np.squeeze(nu_0_L_bar)
    print("Bar shapes: ", mu_0_L_bar.shape, nu_0_L_bar.shape, la_0_L_bar.shape)
    mu_0_U_bar, nu_0_U_bar, la_0_U_bar = get_bar_upper(LUB, mu_0_L, mu_0_U, 
                                                       nu_0_L, nu_0_U,
                                                      la_0_L, la_0_U)
    nu_0_U_bar = np.squeeze(nu_0_U_bar)
    print("Bar shapes: ", mu_0_L_bar.shape, nu_0_L_bar.shape, la_0_L_bar.shape)
    print("Bar shapes: ", mu_0_U_bar.shape, nu_0_U_bar.shape, la_0_U_bar.shape)
    
    z_1_L = my_relu(zeta_0_L)
    z_1_U = my_relu(zeta_0_U)
    
    #Second layer
    W_1_L, W_1_U, b_1_L, b_1_U = sW_1 - dW_1*w_margin,  sW_1 + dW_1*w_margin, sb_1 - db_1*w_margin, sb_1 + db_1*w_margin
    
    print("In Shapes: ", W_1_L.shape, mu_0_L_bar.shape, nu_0_L_bar.shape, la_0_L_bar.shape,
                                   mu_0_U_bar.shape, nu_0_U_bar.shape, la_0_U_bar.shape)
    
    a_L, b_L, c_L = get_abc_lower(W_1_L, mu_0_L_bar, nu_0_L_bar, la_0_L_bar,
                                   mu_0_U_bar, nu_0_U_bar, la_0_U_bar)

    a_U, b_U, c_U = get_abc_upper(W_1_U, mu_0_L_bar, nu_0_L_bar, la_0_L_bar,
                           mu_0_U_bar, nu_0_U_bar, la_0_U_bar)
    
    print("ABC shapes: ", a_L.shape, b_L.shape, c_L.shape)
    print("ABC shapes: ", a_U.shape, b_U.shape, c_U.shape)
    
    mu_1_l = np.sum(a_L, axis=0); mu_1_u = np.sum(a_U, axis=0)
    nu_12_l = z_1_L; 
    nu_12_u = z_1_L
    nu_02_l = b_L; 
    nu_02_u = b_U
    #la_1_l = np.sum(c_L, axis=0) - np.dot(z_1_L, W_1_L) + b_1_L
    #la_1_u = np.sum(c_U, axis=0) - np.dot(z_1_L, W_1_U) + b_1_U
    la_1_l = np.sum(c_L) - np.dot(z_1_L, W_1_L) + b_1_L
    la_1_u = np.sum(c_U) - np.dot(z_1_L, W_1_U) + b_1_U
    la_1_l = np.squeeze(la_1_l)
    la_1_u = np.squeeze(la_1_u)
    
    mu_1_l = np.asarray([mu_1_l])
    print("---------")
    print(mu_1_l.shape, b_L.shape, z_1_L.shape)
    print(np.asarray(x_u).shape, W_0_U.shape, W_1_U.shape)
    
    
    out_l = min_of_linear_fun(np.concatenate((mu_1_l.flatten(), b_L.flatten(), z_1_L.flatten())), 
                                np.concatenate((np.asarray(x_u).flatten(), W_0_U.flatten(), W_1_U.flatten())),
                                np.concatenate((np.asarray(x_l).flatten(), W_0_L.flatten(), W_1_L.flatten()))) + la_1_l


    mu_1_u = np.asarray([mu_1_u])
    out_u = max_of_linear_fun(np.concatenate((mu_1_u.flatten(), nu_02_u.flatten(), nu_12_u)), 
                                np.concatenate((np.asarray(x_u).flatten(), W_0_U.flatten(), W_1_U.flatten())),
                                np.concatenate((np.asarray(x_l).flatten(),W_0_L.flatten(), W_1_L.flatten()))) + la_1_u

    #print out_l[0], out_u[0]
    #print 'range: ' + str(out_u[0] - out_l[0])
    return [sW_0,sb_0,sW_1,sb_1],[out_l, out_u]

In [28]:
# ============
# Full routine
# ============
def prob_veri(model, s0, s1, w_marg, samples, predicate, i0=0, depth=4):
    assert(samples >= (depth)) #, "Ensure samples > depth. Otherwise probability computation is unsound.")
    w_marg = w_marg*2
    safe_weights = []
    for i in trange(samples, desc="Checking Samples"):
        model.model.set_weights(model.sample())
        ol, ou = propogate_lines(model, (s0+s1)/2, [s0, s1], w_margin=w_marg)
        if(predicate(np.squeeze(s0), np.squeeze(s1), np.squeeze(ol), np.squeeze(ou))):
            safe_weights.append(model.model.get_weights())
    print("Found %s safe intervals"%(len(safe_weights)))
    p = compute_probability_bonferroni(model, safe_weights, w_marg, max_depth=depth)
    return p

In [11]:
print("hello")

hello


In [49]:
xl = np.asarray([X_test[0] - 0.01])
xu = np.asarray([X_test[0] + 0.01])

def predicate_safe(iml, imu, ol, ou):
    v1 = tf.one_hot(TRUE_VALUE, depth=10)
    v2 = 1 - tf.one_hot(TRUE_VALUE, depth=10)
    v1 = tf.squeeze(v1); v2 = tf.squeeze(v2)
    worst_case = tf.math.add(tf.math.multiply(v2, ou), tf.math.multiply(v1, ol))
    if(np.argmax(worst_case) == TRUE_VALUE):
        return True
    else:
        return False
    
p = prob_veri(bayes_model, xl, xu, w_marg=3.5, samples=10, predicate=predicate_safe, i0=0, depth=4)

Checking Samples:   0%|                                                                                         | 0/10 [00:00<?, ?it/s]


[-0.07213475 -0.01314103  0.07552157 -0.07011431  0.00535314 -0.0718822
 -0.01322735 -0.07246232  0.07064665 -0.07447143  0.06007013  0.02746662
  0.01272532 -0.0222773  -0.03831224 -0.0486553 ]
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
(1568,) (1568,) (1568,)
Zeta Shapes:  (1, 16) (1, 16)
(16,)
B

IndexError: index 13488 is out of bounds for axis 0 with size 13488